In [1]:

# Set up notebook
%pprint
%matplotlib inline
import sys
import os.path as osp, os as os

executable_path = sys.executable
scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts'); assert osp.exists(scripts_folder)
py_folder = osp.abspath(osp.join(os.pardir, 'py')); assert osp.exists(py_folder), "Create the py folder"
ffmpeg_folder = r'C:\ffmpeg\bin'; assert osp.exists(ffmpeg_folder)
shared_folder = osp.abspath(osp.join(os.pardir, 'share')); assert osp.exists(shared_folder)

if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)

from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join(os.pardir, 'data')),
    saves_folder_path=osp.abspath(osp.join(os.pardir, 'saves'))
)
nu.delete_ipynb_checkpoint_folders()

from pathlib import Path
root = Path(Path(os.sep).resolve().drive + os.sep)

# Import needed libraries
import re
import pandas as pd
import pyperclip
import ipywidgets as widgets
from IPython.display import display
import inspect

Pretty printing has been turned OFF


In [2]:

# Download all the vector and raster map data from Natural Earth
import geopandas as gpd

# Read in the zip file of everything
file_path = osp.abspath(osp.join(nu.data_folder, 'zip', 'ne_10m_admin_0_countries.zip'))
countries_df = gpd.read_file(file_path)

In [17]:

# Get the name of the smallest country by convex bounded area
smallest_country = min(country_areas, key=lambda x: x[1])[0]

# Get the smallest country width and height
mask_series = countries_df.NAME_EN.isin([smallest_country])
for (country_name, geometry), df in countries_df[mask_series].groupby(['NAME_EN', 'geometry']):
    bounds_tuple = geometry.bounds
    
    # Bounds Breakdown:
    smallest_country_minx = bounds_tuple[0]
    smallest_country_miny = bounds_tuple[1]
    smallest_country_maxx = bounds_tuple[2]
    smallest_country_maxy = bounds_tuple[3]
    
    # Rectangle Parameters:
    smallest_country_width = smallest_country_maxx - smallest_country_minx
    smallest_country_height = smallest_country_maxy - smallest_country_miny

    # Print the bits for the whole and loop through the parts
    print(country_name, smallest_country_width, smallest_country_height, bounds_tuple, sep='\t')
    for geom in geometry.geoms:
        bounds_tuple = geom.bounds
        
        # Bounds Breakdown:
        minx = bounds_tuple[0]
        miny = bounds_tuple[1]
        maxx = bounds_tuple[2]
        maxy = bounds_tuple[3]
        
        # Rectangle Parameters:
        width = maxx - minx
        height = maxy - miny

        # Display the bits for each geom in the country
        print(' '*len(country_name), width, height, bounds_tuple, sep='\t')

Bahrain	0.43934560475753415	0.7075478735193883	(50.38052953694196, 25.579890923319237, 50.81987514169949, 26.287438796838625)
       	0.19675597118155963	0.45304362000001674	(50.44890384200005, 25.789536851000037, 50.64565981318161, 26.242580471000053)
       	0.04293607712382652	0.054743498332886276	(50.38052953694196, 26.119988096055685, 50.42346561406578, 26.17473159438857)
       	0.08479875231956413	0.08479875231956413	(50.59950353027349, 26.20264004451906, 50.68430228259305, 26.287438796838625)
       	0.06328429875490826	0.1512494740242225	(50.73760555331811, 25.579890923319237, 50.80088985207302, 25.73114039734346)
       	0.01582107468873062	0.010758330788334547	(50.775576132571054, 25.745062943069538, 50.791397207259784, 25.755821273857872)
       	0.01771960365137204	0.015188231701177557	(50.80215553804812, 25.66405904066326, 50.81987514169949, 25.67924727236444)
       	0.03164214937745413	0.02214950456421505	(50.78506877738429, 25.630518362323162, 50.81671092676174, 25.652

In [18]:

# Get the (convex) bounded height and width of all countries together
from shapely.geometry.polygon import Polygon

# Loop through the geometry of all our countries
polygons = []
mask_series = countries_df.NAME_EN.isin(country_names)
for country_multipolygon in countries_df[mask_series].geometry:

    # If the country's geometry is just one geom, append it to the list
    if isinstance(country_multipolygon, Polygon):
        polygons.append(country_multipolygon)

    # Otherwise, use the geoms iterator to break up the mult-polygon into individual geoms
    else:
        polygons.extend(country_multipolygon.geoms)

# Create a new geom from the individual polygons
countries_multipolygon = MultiPolygon(polygons)

# Get the bounds of the new geom
countries_bounds = countries_multipolygon.bounds

# Bounds Breakdown:
countries_minx = countries_bounds[0]
countries_miny = countries_bounds[1]
countries_maxx = countries_bounds[2]
countries_maxy = countries_bounds[3]

# Rectangle Parameters:
countries_width = countries_maxx - countries_minx
countries_height = countries_maxy - countries_miny

print('All Countries', countries_width, countries_height, countries_bounds, sep='\t')

All Countries	38.6312854009999	29.987337970999988	(24.688342732000137, 12.111443672000064, 63.31962813300004, 42.09878164300005)


In [19]:

# Get a data frame of all unit-pixels of the map grid and their polygons
import math
from pandas import DataFrame

# Get the lower left-hand corner of the map
smallest_country_unit = math.ceil(max(smallest_country_width, smallest_country_height))
all_unit = math.ceil(max(countries_width, countries_height))
all_count = int(all_unit/smallest_country_unit)
countries_llc = (math.floor(countries_minx), math.floor(countries_miny))

# Loop through the x- and y-units and build the rows list of the data frame
polygons = []
rows_list = []
for x_unit in range(1, all_count+1):
    for y_unit in range(1, all_count+1):

        # Get all four corners of the grid cell to build the linear ring of the polygon
        llc = (countries_llc[0]+x_unit*smallest_country_unit, countries_llc[1]+y_unit*smallest_country_unit)
        lrc = (countries_llc[0]+(x_unit+1)*smallest_country_unit, countries_llc[1]+y_unit*smallest_country_unit)
        ulc = (countries_llc[0]+x_unit*smallest_country_unit, countries_llc[1]+(y_unit+1)*smallest_country_unit)
        urc = (countries_llc[0]+(x_unit+1)*smallest_country_unit, countries_llc[1]+(y_unit+1)*smallest_country_unit)
        linear_ring = (llc, ulc, urc, lrc, llc)

        # Create the polygon and add it to the row at its xy coordinates
        polygon = Polygon(shell=linear_ring)
        polygons.append(polygon)
        rows_list.append({
            'x_unit': x_unit,
            'y_unit': y_unit,
            'polygon': polygon,
        })

# Create the mult-polygon and data frame
all_polygons = MultiPolygon(polygons)
all_polygons_df = DataFrame(rows_list)

In [20]:

# Get the area of each country in a dictionary
area_dict = {}

# For each country, get the coordinates of the grid it intersects with
mask_series = countries_df.NAME_EN.isin(country_names)
rows_list = []
for (country_name, geometry), df in countries_df[mask_series].groupby(['NAME_EN', 'geometry']):

    # Get the largest polygon
    geometry = nu.get_largest_polygon(geometry)

    area_dict[country_name] = geometry.area
    for row_index, row_series in all_polygons_df.iterrows():
        x_unit = row_series.x_unit
        y_unit = row_series.y_unit
        polygon = row_series.polygon
        if polygon.intersects(geometry):
            # print(country_name, polygon.bounds, 'intersects', geometry.bounds)
            rows_list.append({
                'x_unit': x_unit,
                'y_unit': y_unit,
                'country_name': country_name,
            })
intersections_df = DataFrame(rows_list)

In [21]:

# Give precedence to the smallest country
srs = intersections_df.groupby('country_name').size().sort_values()
assert not (srs == 1).any(), f"{smallest_country} must been given precedence"

# Remove the intersections that compete with the smallest country
precedence_mask = (intersections_df.country_name == smallest_country)
remove_indices = []
for row_index, row_series in intersections_df[precedence_mask].iterrows():
    x_unit = row_series.x_unit
    y_unit = row_series.y_unit
    mask_series = (intersections_df.x_unit == x_unit) & (intersections_df.y_unit == y_unit) & (intersections_df.country_name != smallest_country)
    remove_indices.extend(intersections_df[mask_series].index.tolist())
mask_series = intersections_df.index.isin(remove_indices)
intersections_df = intersections_df[~mask_series]

In [22]:

# Loop through the duplicate intersections
groupby_columns = ['x_unit', 'y_unit']
mask_series = intersections_df.duplicated(subset=groupby_columns, keep=False)
remove_indices = []
for (x_unit, y_unit), df in intersections_df[mask_series].groupby(groupby_columns):

    # Get the country with the largest area in the unit
    mask_series = (all_polygons_df.x_unit == x_unit) & (all_polygons_df.y_unit == y_unit)
    polygon_unit = all_polygons_df[mask_series].polygon.tolist()[0]
    plurality_country = max(df.country_name, key=lambda x: geometry_dict[x].intersection(polygon_unit).area)

    # Mark the losers for removal
    precedence_mask = (intersections_df.country_name == plurality_country)
    mask_series = (intersections_df.x_unit == x_unit) & (intersections_df.y_unit == y_unit) & (intersections_df.country_name != plurality_country)
    remove_indices.extend(intersections_df[mask_series].index.tolist())
    
# Remove the intersection losers by their indices
mask_series = intersections_df.index.isin(remove_indices)
intersections_df = intersections_df[~mask_series]

In [23]:

# Assert that there are no more duplicate intersections
mask_series = intersections_df.duplicated(subset=groupby_columns, keep=False)
assert not mask_series.any()

In [24]:

# Merge the country grid data frame with the polygon grid data frame
merge_df = all_polygons_df.merge(intersections_df, on=groupby_columns, how='left')
merge_df.country_name = merge_df.country_name.fillna('Ocean')
merge_df.sample(5)

,x_unit,y_unit,polygon,country_name
813,21,34,"POLYGON ((45 46, 45 47, 46 47, 46 46, 45 46))",Ocean
628,17,5,"POLYGON ((41 17, 41 18, 42 18, 42 17, 41 17))",Saudi Arabia
970,25,35,"POLYGON ((49 47, 49 48, 50 48, 50 47, 49 47))",Ocean
6,1,7,"POLYGON ((25 19, 25 20, 26 20, 26 19, 25 19))",Ocean
1400,36,36,"POLYGON ((60 48, 60 49, 61 49, 61 48, 60 48))",Ocean


In [25]:

# Assert we still have the same number of countries
assert merge_df.country_name.unique().shape[0] == len(['Ocean'] + country_names)

In [26]:

# Figure out how to get Jordan more convex and contiguous
mask_series = (merge_df.country_name == 'Jordan')
display(merge_df[mask_series])
mask_series = (merge_df.x_unit == 13) & (merge_df.y_unit == 19)
display(merge_df[mask_series])
merge_df.loc[mask_series, 'country_name'] = 'Jordan'

,x_unit,y_unit,polygon,country_name
406,11,17,"POLYGON ((35 29, 35 30, 36 30, 36 29, 35 29))",Jordan
407,11,18,"POLYGON ((35 30, 35 31, 36 31, 36 30, 35 30))",Jordan
408,11,19,"POLYGON ((35 31, 35 32, 36 32, 36 31, 35 31))",Jordan
446,12,18,"POLYGON ((36 30, 36 31, 37 31, 37 30, 36 30))",Jordan
447,12,19,"POLYGON ((36 31, 36 32, 37 32, 37 31, 36 31))",Jordan
485,13,18,"POLYGON ((37 30, 37 31, 38 31, 38 30, 37 30))",Jordan
487,13,20,"POLYGON ((37 32, 37 33, 38 33, 38 32, 37 32))",Jordan
526,14,20,"POLYGON ((38 32, 38 33, 39 33, 39 32, 38 32))",Jordan


,x_unit,y_unit,polygon,country_name
486,13,19,"POLYGON ((37 31, 37 32, 38 32, 38 31, 37 31))",Saudi Arabia


In [224]:

nu.store_objects(xy_unit_country_name_polygons_df=merge_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\xy_unit_country_name_polygons_df.pkl
